In [ ]:
#| default_exp inspecttools

# inspecttools
> Solveit tools for inspecting the symbol table and importing modules

This module provides **LLM tools** for Solveit to dynamically inspect source code, types, and module capabilities. Functions take *string* arguments (dotted symbol paths) rather than Python objects because LLM tool interfaces can only pass serializable values—not live Python references.

In [ ]:
#| export
import inspect, re, sys, ast, builtins, os, linecache
from importlib import import_module
from dialoghelper import add_msg

In [ ]:
from IPython.display import display,Markdown
import textwrap

In [ ]:
#| export
def _find_frame_dict(var:str):
    "Find the dict (globals or locals) containing var"
    frame = inspect.currentframe().f_back
    while frame:
        if var in frame.f_globals: return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [ ]:
#| export
def importmodule(
    mod: str # The module to import (e.g. 'torch.nn.functional')
):
    """Import a module into the caller's global namespace so it's available for `symsrc`, `getval`, `getdir`, etc.
    Use this before inspecting or using symbols from modules not yet imported."""
    g = _find_frame_dict('__msg_id')
    import_module(mod)
    g[mod.split('.')[0]] = import_module(mod.split('.')[0])

`importmodule` lets the LLM dynamically import modules by name. Here we import `fastcore.utils` and verify it's available.

In [ ]:
importmodule('fastcore.utils')
fastcore.__version__

'1.12.2'

In [ ]:
#| export
class SymbolNotFound(Exception): pass

_last = None

def resolve(
    sym: str  # Dotted symbol path, with optional [n] indexing, e.g. "module.attr.subattr[1]" or "_last" for previous result
):
    """Resolve a dotted symbol string to its Python object, with optional [n] indexing.
    Sets global `_last` to the resolved object for chaining.
    Pass `"_last"` to reference the result of the previous tool call.

    Examples:

    - `resolve("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `resolve("mylist[2]")` -> third element of mylist"""
    global _last
    if (sym := sym.strip()) == '_last': return _last
    g = _find_frame_dict('__msg_id')
    if match := re.match(r'^(\w+)\[(\d+)\]$', sym):
        attr, idx = match.groups()
        parts, _last = ['_last'], _last[int(idx)] if attr == '_last' else g[attr][int(idx)]
    else: parts = re.split(r'\.(?![^\[]*\])', sym)
    try: obj = _last if parts[0] == '_last' else g[parts[0]]
    except KeyError: raise SymbolNotFound(f"Symbol '{parts[0]}' not found. Consider using `importmodule` first.")
    for part in parts[1:]:
        match = re.match(r'(\w+)\[(\d+)\]$', part)
        if match:
            attr, idx = match.groups()
            obj = getattr(obj, attr)[int(idx)]
        else: obj = getattr(obj, part)
    _last = obj
    return obj

`resolve` navigates dotted paths like `"a.argfirst"` to reach the actual Python object.

In [ ]:
a = fastcore.utils.L(1)
resolve('a.argfirst')

<bound method L.argfirst of [1]>

It also sets `_last` for chaining. Since it's used internally by all `get*` tools in this module, the tools all set `_last` too.

In [ ]:
_last

<bound method L.argfirst of [1]>

It works on both objects and classes:

In [ ]:
resolve('fastcore.utils.L.argfirst')

<function fastcore.foundation.L.argfirst(self: fastcore.foundation.L, f, negate=False)>

In [ ]:
#| export
def _src_from_lines(lines, start):
    "Extract a single definition from lines starting at start (0-indexed)"
    src = ''.join(lines[start:])
    try: tree = ast.parse(src)
    except SyntaxError: return None
    if not tree.body: return None
    return ''.join(lines[start:start + tree.body[0].end_lineno])

In [ ]:
#| export
def symsrc(
    sym: str  # Dotted symbol path (e.g `Interval` or `sympy.sets.sets.Interval`) or "_last" for previous result
):
    """Get the source code for a symbol.

    Examples:

    - `symsrc("Interval")` -> source code of Interval class if it's already imported
    - `symsrc("sympy.sets.sets.Interval")` -> source code of Interval class
    - `symsrc("_last")` -> source of object from previous tool call
    - For dispatchers or registries of callables: `getnth("module.dispatcher.funcs", n) then symsrc("_last")`"""
    try: obj = resolve(sym)
    except SymbolNotFound as e: return str(e)
    if not (isinstance(obj, type) or callable(obj)): obj = obj.__class__
    try: fname = inspect.getfile(obj)
    except OSError: fname = "<session>"
    try: return f"File: {fname}\n\n{inspect.getsource(obj)}"
    except OSError: pass
    name = obj.__name__
    pat = rf'^(class|def)\s+{name}\b'
    for fname, (_, _, lines, _) in linecache.cache.items():
        src = ''.join(lines)
        if match := re.search(pat, src, re.MULTILINE):
            start = src[:match.start()].count('\n')
            if extracted := _src_from_lines(lines, start): return f"File: {fname}\n\n{extracted}"
    raise OSError(f"Source for {name} not found")

`symsrc` retrieves the source code of any symbol by path—essential for letting the LLM understand how functions work.

In [ ]:
print(symsrc('a.argfirst'))

File: /Users/jhoward/aai-ws/fastcore/fastcore/foundation.py

@patch
@curryable
def argfirst(self:L, f, negate=False):
    "Return index of first matching item"
    if negate: f = not_(f)
    return first(i for i,o in self.enumerate() if f(o))



In [ ]:
class B:
    def a(): ...
b = B()
print(symsrc('b'))

File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_74089/2186209462.py

class B:
    def a(): ...



In [ ]:
def f():
    "testing"
    return 1
print(symsrc('f'))

File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_74089/242320393.py

def f():
    "testing"
    return 1



In [ ]:
#| export
def showsrc(
    sym: str  # Dotted symbol path (e.g `Interval` or `sympy.sets.sets.Interval`) or "_last" for previous result
):
    "Create a note to show the user (and following LLM prompts) the source of `sym`, following `symsrc` rules"
    res = symsrc(sym)
    add_msg(f'```python\n{res}\n```')
    return {'success': 'Message has been added to dialog successfully'}

`showsrc` is like `symsrc` but adds the information as a note message.

In [ ]:
#| export
def gettype(
    sym: str  # Dotted symbol path (e.g `Interval` or `sympy.sets.sets.Interval`) or "_last" for previous result
):
    """Get the type of a symbol and set `_last`.

    Examples:

    - `gettype("sympy.sets.sets.Interval")` -> `<class 'type'>`
    - `gettype("_last")` -> type of previous result"""
    return type(resolve(sym))

`gettype` returns the type of a symbol—useful for the LLM to understand what kind of object it's dealing with.

In [ ]:
gettype('a.argfirst')

method

In [ ]:
gettype('b')

__main__.B

In [ ]:
#| export
def getdir(
    sym: str,  # Dotted symbol path (e.g `Interval` or `sympy.sets.sets.Interval`) or "_last" for previous result
    exclude_private: bool=False # Filter out attrs starting with "_"
):
    """Get dir() listing of a symbol's attributes and set `_last`. E.g: `getdir("sympy.Interval")` -> `['__add__', '__and__', ...]`"""
    res = dir(resolve(sym))
    if not exclude_private: return res
    return [o for o in res if o[0]!='_']

`getdir` lists all attributes of an object (i.e it calls `dir()`. Here we filter out private names to see the public API of an `L` list.

In [ ]:
' '.join(getdir('a', exclude_private=True))

'accumulate append argfirst argwhere attrgot batched clear combinations compress concat copy copy count cycle dropwhile enumerate extend filter flatmap flatten groupby index insert itemgot items map map_dict map_first map_zip map_zipwith pairwise partition permutations pop product range reduce remove renumerate reverse rstarargfirst rstarargwhere rstardropwhile rstarfilter rstarmap rstarpartition rstarreduce rstarsorted rstartakewhile setattrs shuffle sort sorted split splitlines starargfirst starargwhere stardropwhile starfilter starmap starpartition starreduce starsorted startakewhile sum takewhile unique val2idx zip zipwith'

In [ ]:
#| export
def getval(
    sym: str  # Dotted symbol path (e.g `Interval` or `sympy.sets.sets.Interval`) or "_last" for previous result
):
    """Get repr of a symbol's value and set `_last`.

    Examples:
    
    - `getval("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `getval("some_dict.keys")` -> `dict_keys([...])`"""
    return repr(resolve(sym))

`getval` returns the `repr()` of a symbol's value—handy for inspecting data without needing to execute arbitrary code.

In [ ]:
getval('a')

'[1]'

In [ ]:
#| export
def getnth(
    sym: str,  # Dotted symbol path to a dict or object with .values()
    n: int     # Index into the values (0-based)
):
    """Get the nth value from a dict (or any object with .values()). Sets `_last` so you can chain with `symsrc("_last")` etc.

    Examples:
    
    - `getnth("dispatcher.funcs", 12)` -> 13th registered function
    - `getnth("dispatcher.funcs", 0); symsrc("_last")` -> source of first handler"""
    global _last
    _last = list(resolve(sym).values())[n]
    return _last

`getnth` extracts the nth value from a dict (or anything with `.values()`).

In [ ]:
handlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))
getnth('handlers', 0)

<function __main__.<lambda>(x)>

Combined with `_last`, this lets the LLM drill into registries of handlers/dispatchers and then inspect their source.

In [ ]:
symsrc('_last')

'File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_74089/2745627804.py\n\nhandlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))\n'

In [ ]:
#| export
def symlen(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    "Returns the length of the given symbol"
    return len(resolve(sym))

In [ ]:
#| export
def symslice(
    sym: str,   # Dotted symbol path or "_last" for previous result
    start: int, # Starting index for slice
    end: int    # Ending index for slice
):
    "Returns the contents of the symbol from the given start to the end."
    try: return resolve(sym)[start:end]
    except Exception as e: return f'Error: {e}'

In [ ]:
a = ['a', 'b', 'c', 'd']
symslice('a', 1, 3)

['b', 'c']

On failure we get a str error:

In [ ]:
symslice('resolve', 0, 1)

"Error: 'function' object is not subscriptable"

In [ ]:
#| export
def symsearch(
    sym:str,      # Dotted symbol path or "_last" for previous result
    term:str,     # Search term (exact string or regex pattern)
    regex:bool=True,  # If True, regex search; if False, exact match
    flags:int=0   # Regex flags (e.g., re.IGNORECASE)
):
    """Search contents of symbol, which is assumed to be str for regex, or iterable for non-regex.
    Regex mode returns (match, start, end) tuples; otherwise returns (item, index) tuples"""
    if regex: return str([(m.group(), m.start(), m.end()) for m in re.finditer(term, resolve(sym), flags)])
    else: return str([(x, i) for i, x in enumerate(resolve(sym)) if x == term])

In [ ]:
symsearch('a', 'c', regex=False), symsearch('a', 'z', regex=False)

("[('c', 2)]", '[]')

In [ ]:
text = "The quick brown fox jumps over 3 lazy dogs and 12 cats"

In [ ]:
symsearch('text', r'\d+', regex=True)

"[('3', 31, 32), ('12', 47, 49)]"

In [ ]:
symsearch('text', r'\b[aeiou]\w*', regex=True, flags=re.IGNORECASE)

"[('over', 26, 30), ('and', 43, 46)]"

In [ ]:
#| export
def symset(
    val: str  # Value to assign to _ai_sym
):
    "Set _ai_sym to the given value"
    _find_frame_dict('__msg_id')['_ai_sym'] = val

In [ ]:
symset('Otters are awesome!'); _ai_sym

'Otters are awesome!'

In [ ]:
#| export
def run_code_interactive(
    code:str # Code to have user run
):
    """Insert code into user's dialog and request for the user to run it. Use other tools where possible, 
    but if they can not find needed information, *ALWAYS* use this instead of guessing or giving up.
    IMPORTANT: This tool is TERMINAL - after calling it, you MUST stop all tool usage 
    and wait for user response. Never call additional tools after this one."""
    add_msg('# Please run this:\n'+code, msg_type='code')
    return {'success': "CRITICAL: Message added to user dialog. STOP IMMEDIATELY. Do NOT call any more tools. Wait for user to run code and respond."}

### function tracing

`tracetool` is an LLM tool that traces function execution and captures variable snapshots after each line runs, using Python 3.12's `sys.monitoring` for low-overhead tracing.

For each call to the target function (including recursive calls), the LLM receives a stack trace showing how that call was reached, plus a per-line trace dict mapping each executed source line to its hit count and variable snapshots. Variables that don't change are shown as a single `(type, repr)` tuple, while those that evolve across iterations appear as a list of snapshots.

This is particularly useful when the LLM needs to understand unfamiliar code by seeing exactly what happens step-by-step, debug issues by watching how variables change, verify loop behavior by confirming iteration counts and accumulator values, or explore recursive functions where each call gets its own trace entry.

In [ ]:
#| export
_builtins = set(dir(builtins))

def _collapse(v): return v[0] if len(v) > 0 and all(x == v[0] for x in v) else v
def _process(vars): return {k: _collapse(v) for k,v in vars.items() if k not in _builtins}

In [ ]:
#| export
def tracetool(
    sym: str,  # Dotted symbol path of callable to run
    args: list=None,  # Positional args for callable (JSON values passed directly)
    kwargs: dict=None,  # Keyword args for callable (JSON values passed directly)
    target_func: str=None  # Dotted symbol path of function to trace (defaults to sym)
)->list[tuple[str, dict[str, tuple[int, dict[str, tuple|list]]]]]:  # List of (stack_str,trace_dict); trace_dict maps source snippets to (hit_count, variables), unchanged vars collapsed to single tuple
    """
    Trace execution using sys.monitoring (Python 3.12+), returning a list of per-call traces.

    Return:
    - list of length <= 10
    - one element per call to `target_func` (including recursion)
    - each element is: (stack_str, trace_dict)
        stack_str: call stack string (filtered so `fn` is the shallowest frame shown)
        trace_dict: {
        "<source snippet for AST-line>": ( hit_count, { "var": [ (type_name, truncated_repr), ... up to 10 ], ... } ),
        ...}

    Semantics:
    - "Line" means an AST-level line: separate statements (even if on one physical line via `;`).
    - Compound statements are keyed by their header only.
    - Unchanged variables → `('type', 'repr')` tuple, changed variables → `[('type', 'repr'), ...]` list.
    - Comprehensions are treated as a line node and are monitored, including inside the comprehension frame, with per-iteration snapshots.
    - Snapshots are recorded after each line finishes, so assignments show updated values.
    """
    from tracefunc import tracefunc
    target = resolve(target_func) if target_func else None
    traces = tracefunc(resolve(sym), *(args or []), target_func=target, **(kwargs or {}))
    return [(stack, {src: (hits, _process(vars)) for src, (hits, vars) in d.items()}) for stack, d in traces]

In [ ]:
def demo(n,m='x'):
    total = 0
    for i in range(n): total += i
    return m*total

Try using &`tracetool` to trace `demo(5, m='y')`. Based on the results, tell me how `demo` works.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>tracetool(sym=demo, args=[5], kwargs={'m': 'y'})</summary>

```json
{
  "id": "toolu_01VtStbcHSeTqxdKXAXAf7kL",
  "call": {
    "function": "tracetool",
    "arguments": {
      "sym": "demo",
      "args": "[5]",
      "kwargs": "{'m': 'y'}"
    }
  },
  "result": "[('',\n  {'total = 0': (1, {'total': ('int', '0')}),\n   'for i in range(n):': (6,\n    {'i': [('int', '0'),\n      ('int', '1'),\n      ('int', '2'),\n      ('int', '3'),\n      ('int', '4'),\n      ('int', '4')],\n     'n': ('int', '5')}),\n   'total += i': (5,\n    {'i': [('int', '0'),\n      ('int', '1'),\n      ('int', '2'),\n      ('int', '3'),\n      ('int', '4')],\n     'total': [('int', '0'),\n      ('int', '1'),\n      ('int', '3'),\n      ('int', '6'),\n      ('int', '10')]}),\n   'return m*total': (1, {'m': ('str', \"'y'\"), 'total': ('int', '10')})})]"
}
```

</details>

The trace shows exactly how `demo` works:

1. **`total = 0`** — initializes `total` to 0
2. **`for i in range(n)`** — loops with `i` going 0→1→2→3→4 (hit 6 times: 5 iterations + 1 final check)
3. **`total += i`** — accumulates: 0→1→3→6→10 (the sum 0+1+2+3+4)
4. **`return m*total`** — returns `'y' * 10` = `'yyyyyyyyyy'`

So `demo(n, m)` computes the sum of integers from 0 to n-1 (i.e., `n*(n-1)//2`), then repeats the string `m` that many times. With n=5, the sum is 10, so it returns `m` repeated 10 times.

In [ ]:
def quotefunc(s): return textwrap.wrap('aaa '*10, width=10, subsequent_indent='> ')
tests = 'aaa '*10
quotefunc(tests)

['aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aaa']

Try tracing `textwrap.TextWrapper._wrap_chunks` by calling quotefunc.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>tracetool(sym=quotefunc, args=['test'], target_func=textwrap.TextWrapper._wrap_chunks)</summary>

```json
{
  "id": "toolu_01KmGXSpBR2CCkrVAXaiu6ty",
  "call": {
    "function": "tracetool",
    "arguments": {
      "sym": "quotefunc",
      "args": "['test']",
      "target_func": "textwrap.TextWrapper._wrap_chunks"
    }
  },
  "result": "[('quotefunc (4125899319.py:2)\\nwrap (/Users/jhoward/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/textwrap.py:384)\\nwrap (/Users/jhoward/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/textwrap.py:359)\\n_wrap_chunks (/Users/jhoward/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/textwrap.py:238)',\n  {'\"\"\"_wrap_chunks(chunks : [string]) -> [string]\\n\\n    Wrap a sequence of text chunks and return a list of lines of\\n    length \\'self.width\\' or less.  (If \\'break_long_words\\' is false,\\n    some lines may be longer than <TRUNCATED>"
}
```

</details>

Fascinating! Here's how `_wrap_chunks` works:

**Setup:**
- Initializes empty `lines` list, validates width, reverses `chunks` (so it can pop from the end efficiently)

**Main loop (5 iterations for 5 output lines):**
1. Sets `indent` — empty string for line 1, `'> '` for subsequent lines
2. Calculates available `width` — 10 for line 1, 8 for others (after subtracting indent length)
3. Inner loop pops chunks onto `cur_line`, tracking `cur_len`, until adding next chunk would exceed width
4. Strips trailing whitespace: `['aaa', ' ', 'aaa', ' ']` → `['aaa', ' ', 'aaa']` (cur_len 8→7)
5. Joins with indent: `'' + 'aaa aaa'` or `'> ' + 'aaa aaa'`

**Key observations:**
- `width` is 10 on first line but 8 thereafter (due to 2-char `'> '` indent)
- Each line gets exactly `['aaa', ' ', 'aaa']` = 7 chars, fitting within the available width
- The trailing space is always stripped before joining
- Never hits `_handle_long_word` since no chunk exceeds width

In [ ]:
#| export
def fmt_trace(traces):
    "Format tracetool output as markdown"
    def fmt_val(val):
        if isinstance(val, tuple): return f"`{val[1]}`"
        return " → ".join(f"`{v[1]}`" for v in val)
    def fmt_vars(vars):
        if not vars: return ''
        return '<br>'.join(f"`{k}`: {fmt_val(v)}" for k, v in vars.items())
    def fmt_line(src, hits, vars):
        src_short = src.split('\n')[0][:60]
        if len(src.split('\n')[0]) > 60: src_short += '...'
        return f"| `{src_short}` | {hits} | {fmt_vars(vars)} |"
    parts = []
    for stack, trace in traces:
        if stack: parts.append(f"**Stack:**\n```\n{stack}\n```\n")
        parts.append("| Source | Hits | Variables |\n|--------|------|-----------|")
        parts.extend(fmt_line(src, hits, vars) for src, (hits, vars) in trace.items() if hits > 0)
        parts.append("")
    return '\n'.join(parts)

For a human friendly version, use `fmt_trace` on the result.

In [ ]:
r = tracetool(sym='quotefunc', args=['test'], target_func='textwrap.TextWrapper._wrap_chunks')

In [ ]:
Markdown(fmt_trace(r))

**Stack:**
```
quotefunc (4125899319.py:2)
wrap ($PYTHON_STDLIB/textwrap.py:384)
TextWrapper.wrap ($PYTHON_STDLIB/textwrap.py:359)
TextWrapper._wrap_chunks ($PYTHON_STDLIB/textwrap.py:238)
```

| Source | Hits | Variables |
|--------|------|-----------|
| `lines = []` | 1 | `lines`: `[]` |
| `if self.width <= 0:` | 1 | `self`: `<textwrap.TextWrapper object>` |
| `if self.max_lines is not None:` | 1 | `self`: `<textwrap.TextWrapper object>` |
| `chunks.reverse()` | 1 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` |
| `while chunks:` | 6 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa']` → `[' ', 'aaa', ' ', 'aaa']` → `[]` |
| `cur_line = []` | 5 | `cur_line`: `[]` |
| `cur_len = 0` | 5 | `cur_len`: `0` |
| `if lines:` | 5 | `lines`: `[]` → `['aaa aaa']` → `['aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aaa']` |
| `indent = self.subsequent_indent ` | 4 | `indent`: `'> '`<br>`self`: `<textwrap.TextWrapper object>` |
| `indent = self.initial_indent ` | 1 | `indent`: `''`<br>`self`: `<textwrap.TextWrapper object>` |
| `width = self.width - len(indent)` | 5 | `indent`: `''` → `'> '` → `'> '` → `'> '` → `'> '`<br>`self`: `<textwrap.TextWrapper object>`<br>`width`: `10` → `8` → `8` → `8` → `8` |
| `if self.drop_whitespace and chunks[-1].strip() == '' and lin...` | 5 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa']` → `[' ', 'aaa', ' ', 'aaa']`<br>`lines`: `[]` → `['aaa aaa']` → `['aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aaa']`<br>`self`: `<textwrap.TextWrapper object>` |
| `while chunks:` | 25 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` |
| `l = len(chunks[-1])` | 24 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...`<br>`l`: `3` → `1` → `3` → `1` → `3` → `3` → `1` → `3` → `1` → `3` |
| `if cur_len + l <= width:` | 24 | `cur_len`: `0` → `3` → `4` → `7` → `8` → `0` → `3` → `4` → `7` → `8`<br>`l`: `3` → `1` → `3` → `1` → `3` → `3` → `1` → `3` → `1` → `3`<br>`width`: `10` → `10` → `10` → `10` → `10` → `8` → `8` → `8` → `8` → `8` |
| `cur_line.append(chunks.pop())` | 20 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...`<br>`cur_line`: `['aaa']` → `['aaa', ' ']` → `['aaa', ' ', 'aaa']` → `['aaa', ' ', 'aaa', ' ']` → `['aaa']` → `['aaa', ' ']` → `['aaa', ' ', 'aaa']` → `['aaa', ' ', 'aaa', ' ']` → `['aaa']` → `['aaa', ' ']` |
| `cur_len += l` | 20 | `cur_len`: `3` → `4` → `7` → `8` → `3` → `4` → `7` → `8` → `3` → `4`<br>`l`: `3` → `1` → `3` → `1` → `3` → `1` → `3` → `1` → `3` → `1` |
| `break` | 4 |  |
| `if chunks and len(chunks[-1]) > width:` | 5 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa']` → `[' ', 'aaa', ' ', 'aaa']` → `[]`<br>`width`: `10` → `8` → `8` → `8` → `8` |
| `if self.drop_whitespace and cur_line and cur_line[-1].strip(...` | 5 | `cur_line`: `['aaa', ' ', 'aaa', ' ']`<br>`self`: `<textwrap.TextWrapper object>` |
| `cur_len -= len(cur_line[-1])` | 5 | `cur_len`: `7`<br>`cur_line`: `['aaa', ' ', 'aaa', ' ']` |
| `del cur_line[-1]` | 5 | `cur_line`: `['aaa', ' ', 'aaa']` |
| `if cur_line:` | 5 | `cur_line`: `['aaa', ' ', 'aaa']` |
| `if (self.max_lines is None or` | 5 | `chunks`: `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa'...` → `[' ', 'aaa', ' ', 'aaa', ' ', 'aaa', ' ', 'aaa']` → `[' ', 'aaa', ' ', 'aaa']` → `[]`<br>`cur_len`: `7`<br>`lines`: `[]` → `['aaa aaa']` → `['aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aaa']`<br>`self`: `<textwrap.TextWrapper object>`<br>`width`: `10` → `8` → `8` → `8` → `8` |
| `lines.append(indent + ''.join(cur_line))` | 5 | `cur_line`: `['aaa', ' ', 'aaa']`<br>`indent`: `''` → `'> '` → `'> '` → `'> '` → `'> '`<br>`lines`: `['aaa aaa']` → `['aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aaa']` → `['aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aa...` |
| `return lines` | 1 | `lines`: `['aaa aaa', '> aaa aaa', '> aaa aaa', '> aaa aa...` |


In [ ]:
#| export
def inspect_tool_info():
    add_msg('Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,symlen,symslice,symsearch,symset,run_code_interactive,tracetool]`')

In [ ]:
# inspect_tool_info()

Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,symlen,symslice,symsearch,symset,run_code_interactive]`